In [1]:
import numpy as np 
import os
# import skimage.io as io
# import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as py

import tensorflow as tf

Using TensorFlow backend.


In [2]:
# x = [[[3,2,4][4,5,1]]]
x = np.array([[[[3,2,4],[4,5,1]], [[7,3,5],[4,5,9]]]])
print(x.shape)
print(x)

(1, 2, 2, 3)
[[[[3 2 4]
   [4 5 1]]

  [[7 3 5]
   [4 5 9]]]]


In [3]:
input_ = Input(shape=(2,2,3))

o = UpSampling2D(size = (2,2))(input_)

print(type(o))

with tf.Session() as sess:
    output = sess.run(o, feed_dict={input_:x})
    print(output.shape)
    print(output)

<class 'tensorflow.python.framework.ops.Tensor'>
(1, 4, 4, 3)
[[[[3. 2. 4.]
   [3. 2. 4.]
   [4. 5. 1.]
   [4. 5. 1.]]

  [[3. 2. 4.]
   [3. 2. 4.]
   [4. 5. 1.]
   [4. 5. 1.]]

  [[7. 3. 5.]
   [7. 3. 5.]
   [4. 5. 9.]
   [4. 5. 9.]]

  [[7. 3. 5.]
   [7. 3. 5.]
   [4. 5. 9.]
   [4. 5. 9.]]]]


In [2]:
def U_Net(input_size = (256,256,1)):
    
    inputs_ = Input(input_size)
    
    l_conv1_1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal', 
                     name='l_conv1_1')(inputs_)
    l_conv1_2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal', 
                     name='l_conv1_2')(l_conv1_1)
    l_max_pooling_1 = MaxPooling2D(pool_size=(2, 2), name='l_pooling1')(l_conv1_2)
    # shape
    
    l_conv2_1 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv2_1')(l_max_pooling_1)
    l_conv2_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv2_2')(l_conv2_1)
    l_max_pooling_2 = MaxPooling2D(pool_size=(2, 2), name='l_pooling2')(l_conv2_2)
    # shape
    
    l_conv3_1 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv3_1')(l_max_pooling_2)
    l_conv3_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv3_2')(l_conv3_1)
    l_max_pooling_3 = MaxPooling2D(pool_size=(2, 2), name='l_pooling3')(l_conv3_2)
    # shape
    
    l_conv4_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv4_1')(l_max_pooling_3)
    l_conv4_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                     name='l_conv4_2')(l_conv4_1)
    l_dropout4 = Dropout(0.5, name='l_conv4_dropout')(l_conv4_2)
    l_max_pooling_4 = MaxPooling2D(pool_size=(2, 2), name='l_pooling4')(l_dropout4)

    # shape
    
    ###
    
    c_conv_1 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                   name='c_conv_1')(l_max_pooling_4)
    c_conv_2 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                   name='c_conv_2')(c_conv_1)
    c_dropout = Dropout(0.5, name='c_conv_dropout')(c_conv_2)

    ###
    
    # up-convolution 1
    r_conv1_up_1 = UpSampling2D(size = (2,2), name='r_conv1_up_1')(c_dropout)
    r_conv1_up_2 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv1_up_2')(r_conv1_up_1)
    r_concat1 = concatenate([l_dropout4, r_conv1_up_2], axis = 3, name='r_concat1')
    r_conv1_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv1_2')(r_concat1)
    r_conv1_3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv1_3')(r_conv1_2)

    # up-convolution 2
    r_conv2_up_1 = UpSampling2D(size = (2,2), name='r_conv2_up_1')(r_conv1_3)
    r_conv2_up_2 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv2_up_2')(r_conv2_up_1)
    r_concat2 = concatenate([l_conv3_2, r_conv2_up_2], axis = 3, name='r_concat2')
    r_conv2_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv2_2')(r_concat2)
    r_conv2_3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',
                                  name='r_conv2_3')(r_conv2_2)

    # up-convolution 3
    r_conv3_up_1 = UpSampling2D(size = (2,2), name='r_conv3_up_1')(r_conv2_3)
    r_conv3_up_2 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv3_up_2')(r_conv3_up_1)
    r_concat3 = concatenate([l_conv2_2, r_conv3_up_2], axis = 3, name='r_concat3')
    r_conv3_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv3_2')(r_concat3)
    r_conv3_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv3_3')(r_conv3_2)

    # up-convolution 4
    r_conv4_up_1 = UpSampling2D(size = (2,2), name='r_conv4_up_1')(r_conv3_3)
    r_conv4_up_2 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv4_up_2')(r_conv4_up_1)
    r_concat4 = concatenate([l_conv1_2, r_conv4_up_2], axis = 3, name='r_concat4')
    r_conv4_2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv4_2')(r_concat4)
    r_conv4_3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv4_3')(r_conv4_2)
    
    
    r_conv4_4 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', 
                                  name='r_conv4_4')(r_conv4_3)
    r_conv4_5 = Conv2D(1, 1, activation = 'sigmoid', name='r_conv4_5')(r_conv4_4)

    model = Model(inputs = inputs_, outputs = r_conv4_5)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [3]:
unet = U_Net()
unet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
l_conv1_1 (Conv2D)              (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
l_conv1_2 (Conv2D)              (None, 256, 256, 64) 36928       l_conv1_1[0][0]                  
__________________________________________________________________________________________________
l_pooling1 (MaxPooling2D)       (None, 128, 128, 64) 0           l_conv1_2[0][0]                  
__________________________________________________________________________________________________
l_conv2_1 

In [6]:
plot_model(unet, 
           to_file='./model_images/unet_model_1.png', 
           show_shapes=True, 
           show_layer_names=True)

<img src='./model_images/unet_model_1.png', style='height:2400px'>

In [4]:
def adjust_Data(image, label):
    if np.max(image) > 1:
        image = image/255
        label = label/255
        label[label > 0.5] = 1
        label[label <= 0.5] = 0
    return image, label

In [5]:
def data_generator():
    
    data_gen_args = dict(rotation_range=20,
                         width_shift_range=0.05,
                         height_shift_range=0.05,
                         shear_range=0.05,
                         zoom_range=0.05,
                         horizontal_flip=True,
                         fill_mode='nearest')

    file_path="data/membrane/train"
#     save_to = "data/membrane/train/aug"
    image_color_mode = "grayscale"
    label_color_mode = "grayscale"
    image_folder="image"
    label_folder="label"
    target_size=(256, 256)
    batch_size=2

    # image_datagen = ImageDataGenerator()
    image_datagen = ImageDataGenerator(**data_gen_args)
    image_generator = image_datagen.flow_from_directory(directory = file_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
    #     save_to_dir = save_to,
    #     save_prefix = "image",
        seed=1
    )

    label_datagen = ImageDataGenerator(**data_gen_args)
    label_generator = label_datagen.flow_from_directory(directory = file_path,
        classes = [label_folder],
        class_mode = None,
        color_mode = label_color_mode,
        target_size = target_size,
        batch_size = batch_size,
    #     save_to_dir = save_to,
    #     save_prefix = "label",
        seed=1
    )
    
#     for image in image_generator:
#         plt.imshow(image[0][:, :, 0], cmap='gray')
#         plt.show()
    
#     train_generator = zip(image_generator, label_generator)
#     i = 0
#     for (image, label) in train_generator:
#         if i == 60:
#             break
#         image, label = adjust_Data(image, label)
#         i+=1
#         yield (image, label)   

    train_generator = zip(image_generator, label_generator)
    for (image, label) in train_generator:
        img, lbl = adjust_Data(image, label)
        yield (img, lbl)   

In [20]:
train_data_generator = data_generator()

train_data = []
label_data = []
for image, label in train_data_generator:
    train_data.append(image[0])
    label_data.append(label[0])

print(train_data[0].shape)
print(train_data[1].shape)
print(len(train_data))
print(len(label_data))

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
(256, 256, 1)
(256, 256, 1)
60
60


In [21]:
train_data = np.asarray(train_data)
label_data = np.asarray(label_data)

print(train_data.shape)
print(label_data.shape)

(60, 256, 256, 1)
(60, 256, 256, 1)


In [8]:
# train_data_generator = data_generator()
# 
train_data_generator = data_generator()
unet_model = U_Net()

model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
unet_model.fit_generator(train_data_generator,steps_per_epoch=2000,epochs=1,callbacks=[model_checkpoint], verbose=1)
# unet_model.fit(x=train_data, y=label_data, batch_size=2, epochs=1, steps_per_epoch=2000, callbacks=[model_checkpoint], 
#          validation_split=0.1)

/Users/yankang/anaconda/envs/opencv/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/yankang/anaconda/envs/opencv/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/yankang/anaconda/envs/opencv/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/yankang/anaconda/envs/opencv/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `con

Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
   2/2000 [..............................] - ETA: 10:47:23 - loss: 0.6944 - acc: 0.4959

KeyboardInterrupt: 

## References

1. [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)